In [1]:
import math
import torch
import gpytorch
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
from LBFGS import FullBatchLBFGS
from sklearn.model_selection import GroupKFold
torch.set_default_tensor_type(torch.DoubleTensor)
import gc
from sklearn.preprocessing import StandardScaler
from TCA import kernel,TCA

In [2]:
data=pd.read_csv('QingData_sbp.csv')
#data = pd.read_csv('data_20p_1h_21f.csv')
#data=data.sample(frac=0.5,random_state=1)
patient_list=data['patient_id'].unique()
#print(patient_list)
feature=['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20']
#feature=['HrRR','HrSS','AmFS','AmSN','AmFN','AmFN_FS','AmFN_SN','TmFN','TmNF','TmFN_NF','Tm_FS','Tm_SF','Tm_SN','Tm_FQ','PAT_S','PAT_F','PAT_Q','ArFS','ArSN','ArNF','ArNF_FN']
n_devices = 2#torch.cuda.device_count()
print('Planning to run on {} GPUs.'.format(n_devices))

Planning to run on 2 GPUs.


In [ ]:
MAE=0
ME=0
STD=0
F=5
X=data[feature]#定义源域训练集
y=data['SBP']
scaler=StandardScaler()
X=scaler.fit_transform(X)
groups=np.array(data['patient_id'])
gkf=GroupKFold(n_splits=F)
i=0
for train_index , test_index in gkf.split(X, y, groups):
    i+=1
    #print("TRAIN:", np.unique(groups[train_index]))
    print("TEST:", np.unique(groups[test_index]))
    train_x,train_y=X[train_index],y.iloc[train_index]
    test_x, test_y = X[test_index],y.iloc[test_index]
#     tca=TCA(kernel_type='linear', dim=5, lamb=1, gamma=1)
#     train_x, test_x=tca.fit(train_x, test_x)
    
    train_x=torch.from_numpy( np.array(train_x) )
    train_y=torch.from_numpy( np.array(train_y) )
    test_x=torch.from_numpy( np.array(test_x) )
    test_y=torch.from_numpy( np.array(test_y))
    
    output_device = torch.device('cuda:0')
    train_x, train_y = train_x.to(output_device), train_y.to(output_device)
    test_x, test_y = test_x.to(output_device), test_y.to(output_device)
    
    class ExactGPModel(gpytorch.models.ExactGP):
        def __init__(self, train_x, train_y, likelihood, n_devices):
            super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
            self.mean_module = gpytorch.means.ConstantMean()
            base_covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.MaternKernel(nu=0.5))

            self.covar_module = gpytorch.kernels.MultiDeviceKernel(
                base_covar_module, device_ids=range(n_devices),
                output_device=output_device
            )

        def forward(self, x):
            mean_x = self.mean_module(x)
            covar_x = self.covar_module(x)
            return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

    def train(train_x,
              train_y,
              n_devices,
              output_device,
              checkpoint_size,
              preconditioner_size,
              n_training_iter,
    ):
        likelihood = gpytorch.likelihoods.GaussianLikelihood().to(output_device)
        model = ExactGPModel(train_x, train_y, likelihood, n_devices).to(output_device)
        model.train()
        likelihood.train()

        optimizer = FullBatchLBFGS(model.parameters(), lr=0.1)
        # "Loss" for GPs - the marginal log likelihood
        mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)


        with gpytorch.beta_features.checkpoint_kernel(checkpoint_size), \
             gpytorch.settings.max_preconditioner_size(preconditioner_size):

            def closure():
                optimizer.zero_grad()
                output = model(train_x)
                loss = -mll(output, train_y)
                return loss

            loss = closure()
            loss.backward()

            for i in range(n_training_iter):
                options = {'closure': closure, 'current_loss': loss, 'max_ls': 10}
                loss, _, _, _, _, _, _, fail = optimizer.step(options)

#                 print('Iter %d/%d - Loss: %.3f   lengthscale: %.3f   noise: %.3f' % (
#                     i + 1, n_training_iter, loss.item(),
#                     model.covar_module.module.base_kernel.lengthscale.item(),
#                     model.likelihood.noise.item()
#                 ))

                if fail:
                    print('Convergence reached!')
                    break

        print(f"Finished training on {train_x.size(0)} data points using {n_devices} GPUs.")
        return model, likelihood
    
    def find_best_gpu_setting(train_x,
                              train_y,
                              n_devices,
                              output_device,
                              preconditioner_size
    ):
        N = train_x.size(0)

        # Find the optimum partition/checkpoint size by decreasing in powers of 2
        # Start with no partitioning (size = 0)
        settings = [0] + [int(n) for n in np.ceil(N / 2**np.arange(1, np.floor(np.log2(N))))]

        for checkpoint_size in settings:
            print('Number of devices: {} -- Kernel partition size: {}'.format(n_devices, checkpoint_size))
            try:
                # Try a full forward and backward pass with this setting to check memory usage
                _, _ = train(train_x, train_y,
                             n_devices=n_devices, output_device=output_device,
                             checkpoint_size=checkpoint_size,
                             preconditioner_size=preconditioner_size, n_training_iter=1)

                # when successful, break out of for-loop and jump to finally block
                break
            except RuntimeError as e:
                print('RuntimeError: {}'.format(e))
            except AttributeError as e:
                print('AttributeError: {}'.format(e))
            finally:
                # handle CUDA OOM error
                gc.collect()
                torch.cuda.empty_cache()
        return checkpoint_size

    # Set a large enough preconditioner size to reduce the number of CG iterations run
    preconditioner_size = 100
    checkpoint_size = find_best_gpu_setting(train_x, train_y,
                                            n_devices=n_devices,
                                            output_device=output_device,
                                            preconditioner_size=preconditioner_size)
    
    
    model, likelihood = train(train_x, train_y,
                              n_devices=n_devices, output_device=output_device,
                              checkpoint_size=checkpoint_size,
                              preconditioner_size=preconditioner_size,
                              n_training_iter=100)
    
    model.eval()
    likelihood.eval()
    with torch.no_grad(),\
         gpytorch.settings.fast_pred_var(),\
         gpytorch.beta_features.checkpoint_kernel(checkpoint_size):        
        y_preds = likelihood(model(test_x))
        y_mean=y_preds.mean
    
    
    prediction_data={'ID':groups[test_index],
                     '真实值': test_y.cpu(),
                     '预测值': y_mean.cpu()}
    prediction_data=pd.DataFrame(prediction_data) 
    prediction_data.to_csv('fold '+ '%d' %i +'.csv')
        
    mae=mean_absolute_error(test_y.cpu(),y_mean.cpu())
    MAE+=mae
    error=(test_y.cpu()-y_mean.cpu()).numpy()
    me=np.mean(error)
    ME+=me
    std=np.std(error,ddof=1)
    STD+=std*(test_x.cpu().shape[0]-1)
    print('Validation on fold',i, 'MAE: ', format(mae,'.3f'),'ME',format(me,'.3f'),'STD: ',format(std,'.3f'))
    print('  ')

print('average MAE: ', format(MAE/F,'.3f'),'ME: ', format(ME/F,'.3f'), 'STD: ', format(STD/(data.shape[0]-F),'.3f'))

TEST: [ 0  9 15 16 21 27 29 30 37 44 50 53 61]
